In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# CELDA 3: Preprocesamiento de datos y creación del modelo

# Convertir variables categóricas a numéricas
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')

# Dividir los datos en características (X) y etiquetas (y)
X = dataset.drop('expenses', axis=1)
y = dataset['expenses']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
from sklearn.model_selection import train_test_split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (escalar las características para que tengan media 0 y desviación estándar 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

# Crear el modelo
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset[0])]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='mae',  # Error absoluto medio
              metrics=['mae', 'mse'])  # Métricas a monitorear

# Entrenar el modelo
history = model.fit(
    train_dataset, train_labels,
    epochs=100,  # Número de épocas
    validation_split=0.2,  # Usar 20% de los datos de entrenamiento como validación
    verbose=0,  # No mostrar el progreso de cada época
    callbacks=[tfdocs.modeling.EpochDots()]  # Mostrar puntos en cada época
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
